In [1]:
#pip install seaborn
# !pip install openpyxl

In [48]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline


In [49]:
# for dirname, _, filenames in os.walk(r'..\data'):
#     dir_name=dirname
dir_name="data"

In [4]:
# dir_name=os.path.join('../','data_1')

# Reading Data files


In [50]:
df_1=pd.read_excel(os.path.join(dir_name,'cl_sell_buy.xlsx'),engine='openpyxl',)
df_2=pd.read_excel(os.path.join(dir_name,'diffs.xlsx'),engine='openpyxl',)  
df_3=pd.read_excel(os.path.join(dir_name,'margins.xlsx'),engine='openpyxl',)

In [51]:
def delete_cols(col_names_list,date_list):
    for name in df_lag.columns:
            if 't-' in name or name in date_list or name=='action':
                pass
            else:
#                 print ('column name in else is:',name)
                del df_lag[name]

In [52]:
def input_output(col_names_list):
    for name in df_lag.columns:
            if 'action' in name:
                del df_input[name]
            else:
#                 print ('column name in else is:',name)
                pass

In [53]:
def normalize_df(df):

    # copy the data
    df_min_max_scaled = df.copy()

    # apply normalization techniques
    for column in df_min_max_scaled.columns:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    

    # view normalized data
    return(df_min_max_scaled)

In [54]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [55]:
start_date = '2015-01-10'

In [56]:
mask = (df_1['date'] > start_date)
df_1 = df_1.loc[mask]
df_1.dropna(axis=0, inplace=True)


In [57]:
df_1.head()

,date,time_spread,dod_change,action
1015,2015-01-12,-0.69,-0.06,SELL
1016,2015-01-13,-0.62,0.07,BUY
1017,2015-01-14,-0.48,0.14,BUY
1018,2015-01-15,-0.48,0.00,BUY
1019,2015-01-16,-0.44,0.04,BUY


In [58]:
df_2.head()

,ObservationDate,MEH-WTI,MID,LLS
0,2015-10-01,0.25,-0.20,2.25
1,2015-10-02,-0.40,-0.20,1.80
2,2015-10-05,-0.45,-0.25,1.60
3,2015-10-06,0.10,-0.20,1.65
4,2015-10-07,0.00,-0.05,2.00


In [63]:
mask = (df_2['ObservationDate'] > start_date)
df_2 = df_2.loc[mask]
df_2.dropna(axis=0, inplace=True)

In [64]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384 entries, 0 to 1383
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ObservationDate  1384 non-null   datetime64[ns]
 1   MEH-WTI          1384 non-null   float64       
 2   MID              1384 non-null   float64       
 3   LLS              1384 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 54.1 KB


In [16]:
df_3.head()

,DateCalculated,Contract,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK
0,2013-01-22,2013-02-01,NaN,NaN,5.78,9.29
1,2013-04-22,2013-05-01,NaN,NaN,18.92,19.77
2,2013-05-21,2013-06-01,NaN,NaN,15.99,16.87
3,2013-07-22,2013-08-01,NaN,NaN,-0.67,0.96
4,2013-10-21,2013-11-01,NaN,NaN,1.48,4.42


In [17]:
mask = (df_3['DateCalculated'] > start_date)
df_3 = df_3.loc[mask]
df_3.dropna(axis=0, inplace=True)
df_3.describe()

,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK
count,715.000000,715.000000,715.000000,715.000000
mean,2.555455,0.997818,3.782629,5.967385
std,1.995024,2.077284,3.588861,3.573026
min,-1.530000,-3.810000,-3.270000,-1.240000
25%,1.085000,-0.530000,1.275000,3.505000
50%,2.350000,0.820000,3.320000,5.710000
75%,3.840000,2.180000,6.380000,8.395000
max,16.540000,14.740000,57.230000,60.230000


In [18]:
df1_df2=pd.merge(
    df_1,
    df_2,
    left_on=['date'],
    right_on=['ObservationDate']
)

In [19]:
combined_df=pd.merge(
    df1_df2,
    df_3,
    left_on=['date'],
    right_on=['DateCalculated']
)


In [20]:
combined_df['year'] = pd.DatetimeIndex(combined_df['date']).year
combined_df['month'] = pd.DatetimeIndex(combined_df['date']).month
combined_df['day'] = pd.DatetimeIndex(combined_df['date']).day
combined_df['weekday'] = pd.DatetimeIndex(combined_df['date']).weekday   # Monday is 0 and Sunday is 6

In [21]:
combined_df=combined_df.drop(columns=['date','ObservationDate','DateCalculated', 'Contract'])

In [22]:
lags = range(1, 21)  # 5 lags

df_lag=combined_df.assign(**{'{} (t-{})'.format(col, t): combined_df[col].shift(t)
    for t in lags
    for col in combined_df
})

In [23]:
df_lag.head()

,time_spread,dod_change,action,MEH-WTI,MID,LLS,MEH | USGC | HSK,MEH | USGC | FCC,WTI | P2 | FCC,WTI | P2 | HSK,...,MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20),year (t-20),month (t-20),day (t-20),weekday (t-20)
0,-1.07,0.26,BUY,-0.50,0.55,1.80,0.87,1.76,1.91,0.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.91,0.20,BUY,2.00,0.20,2.75,0.54,0.85,1.33,1.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.61,0.09,BUY,1.85,-0.50,2.35,-1.33,0.28,2.73,2.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.00,-0.41,SELL,0.80,-0.60,1.85,1.28,2.00,3.47,3.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.36,0.23,BUY,1.65,0.00,1.50,1.60,1.74,5.78,5.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#input_df=combined_df.drop(columns=['action','ObservationDate','DateCalculated', 'Contract'])

In [25]:
print (df_lag.columns)

Index(['time_spread', 'dod_change', 'action', 'MEH-WTI', 'MID', 'LLS',
       'MEH | USGC | HSK', 'MEH | USGC | FCC', 'WTI | P2 | FCC',
       'WTI | P2 | HSK',
       ...
       'MID (t-20)', 'LLS (t-20)', 'MEH | USGC | HSK (t-20)',
       'MEH | USGC | FCC (t-20)', 'WTI | P2 | FCC (t-20)',
       'WTI | P2 | HSK (t-20)', 'year (t-20)', 'month (t-20)', 'day (t-20)',
       'weekday (t-20)'],
      dtype='object', length=294)


In [26]:
dates_list=['year','month','weekday','day']

In [27]:
delete_cols(df_lag.columns,dates_list)

In [28]:

#             df_lag.drop(labels=name, axis=1)
            #pass
            

In [29]:
df_lag.head()

,action,year,month,day,weekday,time_spread (t-1),dod_change (t-1),action (t-1),MEH-WTI (t-1),MID (t-1),...,MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20),year (t-20),month (t-20),day (t-20),weekday (t-20)
0,BUY,2015,12,21,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BUY,2016,2,22,0,-1.07,0.26,BUY,-0.50,0.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BUY,2016,3,21,0,-1.91,0.20,BUY,2.00,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SELL,2016,6,21,1,-1.61,0.09,BUY,1.85,-0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BUY,2016,8,22,0,-1.00,-0.41,SELL,0.80,-0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
print(df_lag.isnull().values.sum())

2940


In [31]:
print(df_lag.isnull().sum())

action                    0
year                      0
month                     0
day                       0
weekday                   0
                         ..
WTI | P2 | HSK (t-20)    20
year (t-20)              20
month (t-20)             20
day (t-20)               20
weekday (t-20)           20
Length: 285, dtype: int64


In [32]:
df_lag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704 entries, 0 to 703
Columns: 285 entries, action to weekday (t-20)
dtypes: float64(260), int64(4), object(21)
memory usage: 1.5+ MB


In [33]:
df_lag = df_lag.dropna()

In [34]:
df_lag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 20 to 703
Columns: 285 entries, action to weekday (t-20)
dtypes: float64(260), int64(4), object(21)
memory usage: 1.5+ MB


In [35]:
# df_input=df_lag.drop(columns='action',axis=1) # since these columns wer
df_input=df_lag.copy()
df_output=df_lag['action']

In [36]:
df_cat = df_input.select_dtypes(include=['object']).copy()

In [37]:
df_cat.head()

,action,action (t-1),action (t-2),action (t-3),action (t-4),action (t-5),action (t-6),action (t-7),action (t-8),action (t-9),...,action (t-11),action (t-12),action (t-13),action (t-14),action (t-15),action (t-16),action (t-17),action (t-18),action (t-19),action (t-20)
20,SELL,SELL,SELL,BUY,BUY,SELL,SELL,BUY,BUY,BUY,...,SELL,SELL,SELL,BUY,SELL,BUY,SELL,BUY,BUY,BUY
21,SELL,SELL,SELL,SELL,BUY,BUY,SELL,SELL,BUY,BUY,...,SELL,SELL,SELL,SELL,BUY,SELL,BUY,SELL,BUY,BUY
22,BUY,SELL,SELL,SELL,SELL,BUY,BUY,SELL,SELL,BUY,...,BUY,SELL,SELL,SELL,SELL,BUY,SELL,BUY,SELL,BUY
23,BUY,BUY,SELL,SELL,SELL,SELL,BUY,BUY,SELL,SELL,...,BUY,BUY,SELL,SELL,SELL,SELL,BUY,SELL,BUY,SELL
24,SELL,BUY,BUY,SELL,SELL,SELL,SELL,BUY,BUY,SELL,...,BUY,BUY,BUY,SELL,SELL,SELL,SELL,BUY,SELL,BUY


In [38]:
#categorical data
categorical_cols = df_cat.columns 


In [39]:
df_input=MultiColumnLabelEncoder(columns = categorical_cols).fit_transform(df_input)    # 0 means buy, 1 means sell

In [40]:
df_n_input=normalize_df(df_input)

In [41]:
df_n_input.head()

,action,year,month,day,weekday,time_spread (t-1),dod_change (t-1),action (t-1),MEH-WTI (t-1),MID (t-1),...,MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20),year (t-20),month (t-20),day (t-20),weekday (t-20)
20,1.0,0.0,0.0,0.500000,0.25,0.747967,0.197248,1.0,0.705882,0.922330,...,0.873786,0.583784,0.162162,0.393362,0.396631,0.146042,0.000000,1.000000,0.666667,0.00
21,1.0,0.0,0.0,0.533333,0.50,0.746951,0.203364,1.0,0.695502,0.912621,...,0.856796,0.635135,0.139865,0.329096,0.352221,0.237510,0.166667,0.090909,0.700000,0.00
22,0.0,0.0,0.0,0.566667,0.75,0.745935,0.203364,1.0,0.695502,0.910194,...,0.822816,0.613514,0.013514,0.288842,0.459418,0.277479,0.166667,0.181818,0.666667,0.00
23,0.0,0.0,0.0,0.600000,1.00,0.746951,0.206422,0.0,0.678201,0.902913,...,0.817961,0.586486,0.189865,0.410311,0.516080,0.343582,0.166667,0.454545,0.666667,0.25
24,1.0,0.0,0.0,0.700000,0.00,0.746951,0.204893,0.0,0.660900,0.895631,...,0.847087,0.567568,0.211486,0.391949,0.692956,0.555726,0.166667,0.636364,0.700000,0.00


In [42]:
# df_num=df_input[~df_input.isin(df_cat)].dropna(axis=1)
# df=df_input.fillna(0)

In [43]:
df_output.head()

20    SELL
21    SELL
22     BUY
23     BUY
24    SELL
Name: action, dtype: object

In [44]:
# frames = [df_n_input, df_output]

# df_final = pd.concat(frames)

In [45]:
df_final=df_input.copy()

In [46]:
df_final.head()


,action,year,month,day,weekday,time_spread (t-1),dod_change (t-1),action (t-1),MEH-WTI (t-1),MID (t-1),...,MID (t-20),LLS (t-20),MEH | USGC | HSK (t-20),MEH | USGC | FCC (t-20),WTI | P2 | FCC (t-20),WTI | P2 | HSK (t-20),year (t-20),month (t-20),day (t-20),weekday (t-20)
20,1,2018,1,16,1,0.07,-0.05,1,4.20,1.55,...,0.55,1.80,0.87,1.76,1.91,0.66,2015.0,12.0,21.0,0.0
21,1,2018,1,17,2,0.06,-0.01,1,4.05,1.35,...,0.20,2.75,0.54,0.85,1.33,1.85,2016.0,2.0,22.0,0.0
22,0,2018,1,18,3,0.05,-0.01,1,4.05,1.30,...,-0.50,2.35,-1.33,0.28,2.73,2.37,2016.0,3.0,21.0,0.0
23,0,2018,1,19,4,0.06,0.01,0,3.80,1.15,...,-0.60,1.85,1.28,2.00,3.47,3.23,2016.0,6.0,21.0,1.0
24,1,2018,1,22,0,0.06,0.00,0,3.55,1.00,...,0.00,1.50,1.60,1.74,5.78,5.99,2016.0,8.0,22.0,0.0


In [47]:
df_final.to_csv('features.csv')